In [1]:
import torch
print(torch.__version__)

2.6.0+cpu


In [2]:
from torchvision import datasets

download_dir = ".datasets"

train_data = datasets.CIFAR10(root = download_dir, download = True, train = True)
test_data = datasets.CIFAR10(root = download_dir, download = True, train = False)

In [3]:
train_data

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: .datasets
    Split: Train

In [4]:
x_train = torch.tensor(train_data.data, dtype=torch.float32) / 255.0
x_train = x_train.permute(0, 3, 1, 2)  # Меняем порядок каналов для формата CHW (channels-first)
y_train = torch.tensor(train_data.targets, dtype=torch.long)

x_val = torch.tensor(test_data.data, dtype=torch.float32) / 255.0
x_val = x_val.permute(0, 3, 1, 2)
y_val = torch.tensor(test_data.targets, dtype=torch.long)

x_train.shape

torch.Size([50000, 3, 32, 32])

In [5]:
classes = train_data.classes
classes

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [6]:
from collections import OrderedDict

model = torch.nn.Sequential(
    OrderedDict([
    ('con1', torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3,3), padding = 'same')),
    ('act1', torch.nn.ReLU()),
    ('pol1', torch.nn.MaxPool2d(kernel_size = 2, stride = 2)),
    ('con2', torch.nn.Conv2d(in_channels=32, out_channels= 64, kernel_size=(3,3), padding = 'same')),
    ('act2', torch.nn.ReLU()),
    ('pol2', torch.nn.MaxPool2d(kernel_size = 2, stride = 2)),
    ('con3', torch.nn.Conv2d(in_channels=64,out_channels=128, kernel_size = (3,3), padding = 'same')),
    ('act3', torch.nn.ReLU()),
    ('flat', torch.nn.Flatten()),
    ('lin1', torch.nn.Linear(in_features = 128*8*8, out_features= 512)),
    ('act4', torch.nn.ReLU()),
    ('drop', torch.nn.Dropout(p=0.5)),
    ('lin2', torch.nn.Linear(in_features = 512, out_features = 10)),
    ]))

model

Sequential(
  (con1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (act1): ReLU()
  (pol1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (con2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (act2): ReLU()
  (pol2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (con3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (act3): ReLU()
  (flat): Flatten(start_dim=1, end_dim=-1)
  (lin1): Linear(in_features=8192, out_features=512, bias=True)
  (act4): ReLU()
  (drop): Dropout(p=0.5, inplace=False)
  (lin2): Linear(in_features=512, out_features=10, bias=True)
)

In [8]:
import torch.optim.sgd


optimizer = torch.optim.SGD(
    model.parameters(),
    lr = 0.01
)

loss_func = torch.nn.CrossEntropyLoss()
epochs = 15

In [9]:
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm_notebook

dataset = torch.utils.data.TensorDataset(x_train, y_train)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

losses = []

for epoch in range(epochs):
    with tqdm_notebook(total=len(dataloader), desc=f"Epoch {epoch+1}") as inner_pbar:
        for batch_x, batch_y in dataloader:
            optimizer.zero_grad()
            y_pred = model(batch_x)
            loss = loss_func(y_pred, batch_y)
            loss.backward()
            optimizer.step()

            losses.append(loss.item())

            # Показываем текущую потерю и прогресс
            inner_pbar.set_postfix({'Batch Loss': f"{loss.item():.4f}"})
            inner_pbar.update(1)

Epoch 1:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 2:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 3:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 4:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 5:   0%|          | 0/782 [00:00<?, ?it/s]

...

Epoch 11:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 12:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 13:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 14:   0%|          | 0/782 [00:00<?, ?it/s]

Epoch 15:   0%|          | 0/782 [00:00<?, ?it/s]

In [10]:
res_predicted = model(x_val)
res_predicted

tensor([[ 0.0467,  0.4474,  0.2869,  ..., -0.8446, -0.2561, -1.0294],
        [ 5.3612,  7.1128, -0.3239,  ..., -4.6679,  7.7825,  4.8796],
        [ 3.7968,  4.6729,  0.0832,  ..., -3.2881,  6.9625,  2.9602],
        ...,
        [-1.6993, -3.8606,  2.0521,  ...,  1.2633, -0.6681, -0.5355],
        [ 1.6416,  4.2214, -0.3205,  ..., -1.6399,  1.4793,  1.3798],
        [-0.8222, -0.0118,  0.7496,  ...,  3.2286, -2.3557, -0.5429]],
       grad_fn=<AddmmBackward0>)

In [11]:
predited_classes = res_predicted.argmax(dim=1)
predited_classes = predited_classes.numpy()
predited_classes

array([3, 8, 8, ..., 5, 1, 7], shape=(10000,))

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(predited_classes, test_data.targets)

0.5299